# WaveNet架构详解与时间序列应用

## 教程概览

WaveNet是DeepMind提出的一种用于音频生成的深度生成模型，其核心思想是使用**因果卷积(Causal Convolution)**和**膨胀卷积(Dilated Convolution)**来高效处理序列数据。

### 核心内容
1. **WaveNet的动机**：为什么需要新的序列模型
2. **因果卷积**：保证时序因果关系
3. **膨胀卷积**：指数级扩大感受野
4. **架构实现**：从零构建WaveNet
5. **应用演示**：时间序列预测

### 关键概念
- **因果性(Causality)**：模型只能依赖过去的信息
- **感受野(Receptive Field)**：影响输出的输入范围
- **膨胀率(Dilation Rate)**：卷积核采样间隔

### WaveNet vs RNN
| 特性 | RNN | WaveNet |
|------|-----|---------|
| 并行化 | 串行处理 | 并行训练 |
| 感受野 | 依赖层数 | 指数增长 |
| 梯度传播 | 易消失/爆炸 | 更稳定 |
| 训练速度 | 较慢 | 较快 |

## 一、因果卷积(Causal Convolution)

### 普通卷积的问题

标准的1D卷积在处理序列时会"看到"未来的信息，违反了时序任务的因果性要求。

### 因果卷积的解决方案

通过在输入序列左侧填充(padding)，确保时刻$t$的输出只依赖于$t$及之前的输入。

**Keras实现：**
```python
Conv1D(filters, kernel_size, padding='causal')
```

**数学表示：**
$$y_t = \sum_{k=0}^{K-1} w_k \cdot x_{t-k}$$

其中$K$是卷积核大小，$w_k$是权重

In [ ]:
"""
演示因果卷积与普通卷积的区别
"""
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print("=" * 70)
print("因果卷积 vs 普通卷积")
print("=" * 70)

# 创建简单的输入序列
sequence = np.array([1, 2, 3, 4, 5]).reshape(1, 5, 1).astype(np.float32)

# 普通卷积（会看到未来）
model_normal = keras.models.Sequential([
    keras.layers.Conv1D(filters=1, kernel_size=3, padding='same', 
                        use_bias=False, input_shape=[None, 1])
])
# 设置权重为[1, 1, 1]便于理解
model_normal.layers[0].set_weights([np.ones((3, 1, 1))])

# 因果卷积（不会看到未来）
model_causal = keras.models.Sequential([
    keras.layers.Conv1D(filters=1, kernel_size=3, padding='causal', 
                        use_bias=False, input_shape=[None, 1])
])
model_causal.layers[0].set_weights([np.ones((3, 1, 1))])

# 计算输出
output_normal = model_normal.predict(sequence, verbose=0)
output_causal = model_causal.predict(sequence, verbose=0)

print(f"\n输入序列: {sequence[0, :, 0]}")
print(f"普通卷积输出: {output_normal[0, :, 0]}")
print(f"因果卷积输出: {output_causal[0, :, 0]}")

print("\n观察：")
print("- 普通卷积：第一个输出=6.0，使用了位置0,1,2的值（包含未来）")
print("- 因果卷积：第一个输出=1.0，只使用了位置0的值（不包含未来）")
print("=" * 70)

## 二、膨胀卷积(Dilated Convolution)

### 感受野问题

普通因果卷积的感受野增长缓慢：
- 1层：感受野 = 2
- 2层：感受野 = 3
- n层：感受野 = n + 1

处理长序列需要非常深的网络。

### 膨胀卷积的优势

通过在卷积核元素之间插入"空洞"，指数级扩大感受野：

**膨胀率(dilation rate)**：控制采样间隔
- dilation_rate=1：普通卷积，采样相邻元素
- dilation_rate=2：跳过1个元素采样
- dilation_rate=4：跳过3个元素采样

**感受野计算：**
- 层1 (dilation=1)：感受野 = 2
- 层2 (dilation=2)：感受野 = 4  
- 层3 (dilation=4)：感受野 = 8
- 层4 (dilation=8)：感受野 = 16

仅4层就能达到16的感受野！

In [ ]:
"""
构建WaveNet模型并应用于时间序列预测
"""
print("\n" + "=" * 70)
print("构建WaveNet模型")
print("=" * 70)

# 生成时间序列数据（复用之前的函数）
np.random.seed(42)

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

# 准备数据
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
x_train, y_train = series[:7000, :n_steps], series[:7000, -1]
x_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -1]
x_test, y_test = series[9000:, :n_steps], series[9000:, -1]

print(f"训练数据: X={x_train.shape}, y={y_train.shape}")

# 构建WaveNet模型
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None, 1])
])

# 添加膨胀因果卷积层
# 膨胀率序列：1, 2, 4, 8（指数增长）
for dilation_rate in (1, 2, 4, 8):
    model.add(keras.layers.Conv1D(
        filters=64, 
        kernel_size=2, 
        dilation_rate=dilation_rate,
        padding='causal',
        activation='relu'
    ))

# 添加1x1卷积层进行特征整合
model.add(keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu'))

# 全局池化或取最后时间步
model.add(keras.layers.GlobalAveragePooling1D())

# 输出层
model.add(keras.layers.Dense(1))

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\nWaveNet模型结构：")
model.summary()

# 计算总感受野
print(f"\n理论感受野: 2^1 + 2^2 + 2^3 + 2^4 = 30 时间步")

# 训练模型
print("\n开始训练...")
history = model.fit(
    x_train, y_train,
    epochs=5,                    # 测试时使用较少epoch
    batch_size=32,
    validation_data=(x_val, y_val),
    verbose=1
)

# 评估
test_loss, test_mae = model.evaluate(x_test, y_test, verbose=0)
print(f"\nWaveNet模型性能:")
print(f"  测试集MSE: {test_loss:.6f}")
print(f"  测试集MAE: {test_mae:.6f}")
print("=" * 70)

## 三、WaveNet总结与应用

### 核心优势

1. **并行训练**
   - RNN必须串行处理，WaveNet可以并行计算所有时间步
   - 训练速度显著提升

2. **长距离依赖**
   - 通过膨胀卷积指数级扩大感受野
   - 4层即可达到16的感受野，RNN需要16层

3. **梯度传播**
   - 卷积的梯度传播路径更短更直接
   - 避免RNN的梯度消失/爆炸问题

4. **可解释性**
   - 感受野明确可计算
   - 卷积滤波器有清晰的物理意义

### 应用场景

- **音频生成**：原始论文的主要应用
- **语音合成**：高质量TTS系统
- **时间序列预测**：金融、气象、交通流量
- **机器翻译**：作为Transformer的前身

### 实现要点

1. 使用`padding='causal'`保证因果性
2. 膨胀率呈指数增长（1, 2, 4, 8, 16...）
3. 可以堆叠多个膨胀块(dilation block)
4. 原始WaveNet使用门控激活和残差连接

### 进一步改进

- 添加残差连接(Residual Connections)
- 使用门控激活单元(Gated Activation)
- 跳跃连接(Skip Connections)汇聚多尺度特征
- 条件生成(Conditional WaveNet)用于控制生成